In [1]:
%env CUDA_VISIBLE_DEVICES = 3

env: CUDA_VISIBLE_DEVICES=3


In [2]:
import os, sys
import numpy as np
import time
import torch

from tqdm.auto import tqdm
from copy import copy, deepcopy

from plotnine import *
import plotnine.options as plotnine_opts

In [3]:
sys.path.append("../nsd_deepdive")
from brain_mapping import *

In [5]:
benchmark = "shared1000_OTC-only"
target_region = "OTC"
benchmark = NSDBenchmark(*benchmark.split("_"))

Now loading the shared1000 image set and the OTC-only voxel set...


In [7]:
model_uid = "torchvision_alexnet_imagenet1k_v1"
model, preprocess = get_deepdive_model(model_uid)

image_loader = get_image_loader(benchmark.image_paths, preprocess)
feature_maps = get_all_feature_maps(model, image_loader, numpy=False)

Feature Extraction (Batch):   0%|          | 0/16 [00:00<?, ?it/s]

In [8]:
sys.path.append(".")
from cuda_srp import *

feature_maps_redux = get_feature_map_srps(feature_maps, device="cuda:0")

SR Projection (Layer):   0%|          | 0/18 [00:00<?, ?it/s]

In [46]:
from torch.utils.data import Dataset, DataLoader


class FeatureMapDataSet(Dataset):
    def __init__(self, feature_maps):
        self.model_layers = [key for key in feature_maps.keys()]
        self.feature_maps = [val for val in feature_maps.values()]

    def __len__(self):
        return len(self.feature_maps)

    def __getitem__(self, idx):
        model_layers = self.model_layers[idx]
        feature_maps = self.feature_maps[idx]
        return model_layers, feature_maps


feature_map_dataset = FeatureMapDataSet(feature_maps_redux)
feature_map_loader = DataLoader(feature_map_dataset, batch_size=1, shuffle=False)

In [10]:
X_check = deepcopy(feature_maps_redux["Linear-1"])
y_check = deepcopy(benchmark.response_data.to_numpy().T)

### Torch Distance Matrices (RDMs)

In [31]:
rdm_comparands = {"scipy": {}, "torch": {}, "cupy": {}, "numpy": {}}
n_samples, n_features = 100, 50

In [32]:
from scipy.spatial import distance as scipy_dist
from scipy.stats import spearmanr, pearsonr


def scipy_distance_matrix(input_data, distance, **kwargs):
    return scipy_dist.squareform(scipy_dist.pdist(input_data, distance, **kwargs))


def scipy_distance_matrix_with_time(input_data, distance, **kwargs):
    start_time = time.time()
    if distance == "spearman":
        distance_matrix = 1 - spearmanr(input_data.T)[0]
    if distance not in ["spearman"]:
        distance_matrix = scipy_distance_matrix(input_data, distance, **kwargs)
    total_time = start_time - time.time()

    print(f"Time taken: {round(total_time, 3)} seconds")
    return distance_matrix

In [33]:
sample_matrix = benchmark.response_data.to_numpy()[:n_samples, :n_features]
for distance in ["euclidean", "correlation", "spearman", "mahalanobis"]:
    print(f"Now computing {distance} with scipy...")
    rdm_comparands["scipy"][distance] = scipy_distance_matrix_with_time(
        sample_matrix, distance
    )

rdm_comparands["scipy"]["pearson"] = rdm_comparands["scipy"]["correlation"]

for metric in ["euclidean", "pearson", "spearman"]:
    print(f"{metric} RDM shape: ", rdm_comparands["scipy"][metric].shape)

Now computing euclidean with scipy...
Time taken: -0.001 seconds
Now computing correlation with scipy...
Time taken: -0.001 seconds
Now computing spearman with scipy...
Time taken: -0.03 seconds
Now computing mahalanobis with scipy...
Time taken: -0.01 seconds
euclidean RDM shape:  (100, 100)
pearson RDM shape:  (100, 100)
spearman RDM shape:  (100, 100)


In [118]:
import numpy as np
import cupy as cp

def backend_convert(data, engine = 'torch'):
    if engine == 'numpy':
        if isinstance(data, np.ndarray):
            return data
        elif isinstance(data, torch.Tensor):
            return data.cpu().numpy()
        else:  # assumes data is a cupy array
            return cp.asnumpy(data)
    elif engine == 'cupy':
        if isinstance(data, cp.ndarray):
            return data
        elif isinstance(data, np.ndarray):
            return cp.asarray(data)
        else:  # assumes data is a torch Tensor
            return cp.fromDlpack(torch.utils.dlpack.to_dlpack(data))
    elif engine == 'torch':
        if isinstance(data, torch.Tensor):
            return data
        elif isinstance(data, np.ndarray):
            return torch.from_numpy(data)
        else:  # assumes data is a cupy array
            return torch.utils.dlpack.from_dlpack(data.toDlpack())

def calc_mahalanobis_rdm(data, engine = 'torch', output='torch', cov_matrix=None, norm = False):
    data = backend_convert(data, engine)
    if engine == 'numpy':
        if cov_matrix is None:
            cov_matrix = np.cov(data, rowvar=False)
        inv_cov_matrix = np.linalg.inv(cov_matrix)
        centered_data = data - np.mean(data, axis=0)
        kernel = centered_data @ inv_cov_matrix @ centered_data.T
        rdm = np.diag(kernel)[:, None] + np.diag(kernel)[None, :] - 2 * kernel
    if engine == 'cupy':
        if cov_matrix is None:
            cov_matrix = cp.cov(data, rowvar=False)
        inv_cov_matrix = cp.linalg.inv(cov_matrix)
        centered_data = data - cp.mean(data, axis=0)
        kernel = centered_data @ inv_cov_matrix @ centered_data.T
        rdm = cp.diag(kernel)[:, None] + cp.diag(kernel)[None, :] - 2 * kernel
    if engine == 'torch':
        if cov_matrix is None:
            cov_matrix = torch.cov(data.T) #note transpose
        inv_cov_matrix = torch.inverse(cov_matrix)
        centered_data = data - torch.mean(data, axis=0)
        kernel = centered_data @ inv_cov_matrix @ centered_data.T
        rdm = torch.diag(kernel).unsqueeze(1) + torch.diag(kernel).unsqueeze(0) - 2 * kernel
    if norm: rdm /= data.shape[0]
    return backend_convert(rdm, output)
    
def calc_pearson_rdm(data, engine='torch', output='torch', norm=False):
    data = backend_convert(data, engine)
    if engine == 'numpy':
        rdm = 1 - np.corrcoef(data)
    if engine == 'torch':
        rdm = 1 - torch.corrcoef(data)
    if norm: rdm =/ data.shape[1]
    return backend_convert(rdm, output)

def calc_spearman_rdm(data, engine='torch', output='torch', norm=False):
    data = backend_convert(data, engine)
    
    def rank_order(data):
        return data.clone().argsort(dim=1).argsort(dim=1)
    
    if engine == 'numpy':
        from scipy.stats import spearmanr
        rdm = 1 - spearmanr(data.T)[0]
    if engine == 'torch':
        rank_input = rank_order(data)
        rdm = 1 - torch.corrcoef(rank_input)
    if norm: rdm /= data.shape[0]
    return backend_convert(rdm, output)
    
def calc_euclidean_rdm(data, engine='torch', output='torch', norm=False):
    data = backend_convert(data, engine)
    if engine == 'numpy':
        rdm = squareform(pdist(data, 'euclidean'))
    if engine == 'torch':
        rdm = torch.cdist(data, data, p=2.0)
        rdm = rdm.fill_diagonal_(0)
    if norm: rdm /= data.shape[0]
    return backend_convert(rdm, output)
    
def calculate_rdm(data, distance = 'pearson', engine = 'torch', output = 'torch'):
    if distance == 'euclidean':
        return calc_euclidean_rdm(data, engine, output)
    if distance == 'pearson':
        return calc_pearson_rdm(data, engine, output)
    if distance == 'spearman':
        return calc_spearman_rdm(data, engine, output)
    if distance == 'mahalanobis':
        return calc_mahalanobis_rdm(data, engine, output)

In [119]:
sample_matrix = benchmark.response_data.to_numpy()[:n_samples, :n_features]
for distance in ["mahalanobis", "pearson", "spearman", "euclidean"]:
    for engine in ["torch", "numpy"]:
        rdm = calculate_rdm(
            sample_matrix, distance=distance, engine=engine, output="numpy"
        )
        rdm_comparands[engine][distance] = rdm
        if not rdm.shape == (n_samples, n_samples):
            print(f"{distance} {engine} RDM shape:", rdm.shape)

for distance in ["spearman", "pearson", "euclidean"]:
    for engine in ["torch", "numpy"]:
        print(
            f"{engine} {distance} allclose with scipy: ",
            np.allclose(
                rdm_comparands["scipy"][distance],
                rdm_comparands[engine][distance],
                atol=1e-05,
            ),
        )

torch spearman allclose with scipy:  True
numpy spearman allclose with scipy:  True
torch pearson allclose with scipy:  True
numpy pearson allclose with scipy:  True
torch euclidean allclose with scipy:  True
numpy euclidean allclose with scipy:  True


In [41]:
sample_matrix = benchmark.response_data.to_numpy()[:n_samples, :n_features]
for engine in tqdm(["numpy", "cupy", "torch"]):
    rdm = calc_rdm_mahalanobis(sample_matrix, engine=engine)
    if not engine in rdm_comparands:
        rdm_comparands[engine] = {}
    if rdm.is_cuda:
        rdm = rdm.to("cpu")
    rdm_comparands[engine]["mahalanobis"] = rdm
    print(
        f"{engine} malahanobis allclose with scipy: ",
        np.allclose(
            rdm_comparands["scipy"]["mahalanobis"] ** 2,
            rdm_comparands[engine]["mahalanobis"],
        ),
    )

  0%|          | 0/3 [00:00<?, ?it/s]

numpy malahanobis allclose with scipy:  True
cupy malahanobis allclose with scipy:  True
torch malahanobis allclose with scipy:  True


In [48]:
device = "cpu"  #'cuda:0' if torch.cuda.is_available() else 'cpu'
for layer_index, (model_layer, feature_map) in enumerate(tqdm(feature_map_loader)):
    rdm = calc_rdm_mahalanobis(feature_map.squeeze())

  0%|          | 0/18 [00:00<?, ?it/s]

In [49]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
for layer_index, (model_layer, feature_map) in enumerate(tqdm(feature_map_loader)):
    feature_map = feature_map.to(device)
    rdm = calc_rdm_mahalanobis(feature_map.squeeze())

rdm = rdm.to("cpu")

  0%|          | 0/18 [00:00<?, ?it/s]

In [117]:
def linccc_distance(x1, x2):
    cor = pearsonr(x1, x2)[0]
    x1_mean = np.mean(x1)
    x2_mean = np.mean(x2)
    x1_var = np.var(x1)
    x2_var = np.var(x2)
    x1_std = np.std(x1)
    x2_std = np.std(x2)

    numerator = 2 * cor * x1_std * x2_std
    denominator = x1_var + x2_var + (x1_mean - x2_mean) ** 2

    return numerator / denominator

In [113]:
sample_matrix = benchmark.response_data.to_numpy()[:n_samples, :n_features]
vector1, vector2 = sample_matrix[0], sample_matrix[5]
print("Pearson distance:", round(1 - pearsonr(vector1, vector2)[0], 3))
print("Lin's CCC distance:", round(1 - linccc_distance(vector1, vector2), 3))

Pearson distance: 0.517
Lin's CCC distance: 0.524


In [115]:
def calc_linccc_rdm(data, engine="torch", output="torch"):
    data = convert_matrix(data, engine)

    if engine == "numpy":
        mean_matrix = data.mean(axis=1)
        var_matrix = data.var(axis=1)
        std_matrix = data.std(axis=1)
        corr_matrix = np.corrcoef(data)

    if engine == "torch":
        mean_matrix = data.mean(dim=1)
        var_matrix = data.var(dim=1)
        std_matrix = data.std(dim=1)
        corr_matrix = torch.corrcoef(data)

    # Calculate Lin's CCC
    numerator = 2 * corr_matrix * std_matrix[:, None] * std_matrix[None, :]
    denominator1 = var_matrix[:, None] + var_matrix[None, :]
    denominator2 = (mean_matrix[:, None] - mean_matrix[None, :]) ** 2
    rdm = 1 - (numerator / (denominator1 + denominator2))

    return convert_matrix(rdm, output)

In [126]:
sample_matrix = benchmark.response_data.to_numpy()[:n_samples, :n_features]
rdm = calc_linccc_rdm(sample_matrix, "torch")
print("Lin's CCC distance:", rdm[0, 5].numpy().round(3))

Lin's CCC distance: 0.524


#### RDM Timing Tests

In [315]:
data = torch.from_numpy(benchmark.response_data.to_numpy())

data = data.to(torch.float32).to("cuda:0")

for i in tqdm(range(5)):
    calculate_rdm(data, engine="torch")

data = data.to(torch.float32).to("cpu")

for i in tqdm(range(5)):
    calculate_rdm(data, engine="torch")

data = data.to(torch.float32).numpy()

for i in tqdm(range(5)):
    calculate_rdm(data, engine="numpy")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [316]:
def calculate_rdm(data, distance="pearson", engine="torch", output="torch"):
    if distance == "euclidean":
        return calc_euclidean_rdm(data, engine, output)
    if distance == "pearson":
        return calc_pearson_rdm(data, engine, output)
    if distance == "spearman":
        return calc_spearman_rdm(data, engine, output)
    if distance == "mahalanobis":
        return calc_mahalanobis_rdm(data, engine, output)
    if distance == "concordance":
        return calc_linccc_rdm(data, engine, output)

In [317]:
data = torch.from_numpy(benchmark.response_data.to_numpy())

data = data.to(torch.float32).to("cuda:0")

for method in ["spearman", "pearson", "euclidean", "mahalanobis", "concordance"]:
    for i in tqdm(range(5), desc=method):
        calculate_rdm(data, method)

data = data.to("cpu")  # remove data from cuda

spearman:   0%|          | 0/5 [00:00<?, ?it/s]

pearson:   0%|          | 0/5 [00:00<?, ?it/s]

euclidean:   0%|          | 0/5 [00:00<?, ?it/s]

mahalanobis:   0%|          | 0/5 [00:00<?, ?it/s]

concordance:   0%|          | 0/5 [00:00<?, ?it/s]

### Torch Distance (RSA)

In [299]:
import torchmetrics

score_types = {
    "spearman": torchmetrics.SpearmanCorrCoef,
    "pearson": torchmetrics.PearsonCorrCoef,
    "concordance": torchmetrics.ConcordanceCorrCoef,
    "ev_score": torchmetrics.ExplainedVariance,
}

In [300]:
def extract_triu(X):
    return X[torch.triu(torch.ones_like(X, dtype=bool), diagonal=1)]


y = torch.from_numpy(benchmark.response_data.to_numpy().T)
X = feature_maps_redux["Linear-1"]

X = X.to(torch.float32).to("cuda:0")
y = y.to(torch.float32).to("cuda:0")

rdm1 = calculate_rdm(y)
rdm2 = calculate_rdm(X)

rdm1_triu = extract_triu(rdm1)
rdm2_triu = extract_triu(rdm2)

for score_type, scorer in score_types.items():
    scorer = scorer().to("cuda:0")
    print(score_type, ":", round(scorer(rdm1_triu, rdm2_triu).item(), 3))

del X, y, rdm1, rdm2, rdm1_triu, rdm2_triu, scorer

spearman : 0.339
pearson : 0.364
concordance : 0.08
ev_score : -0.473


In [307]:
from torchmetrics.functional import spearman_corrcoef, pearson_corrcoef
from torchmetrics.functional import concordance_corrcoef, explained_variance

compare_methods = {
    "spearman": spearman_corrcoef,
    "pearson": pearson_corrcoef,
    "concordance": concordance_corrcoef,
}


def compare_rdms(rdm1, rdm2, method="pearson", device=None, **method_kwargs):
    if device is None:
        rdm2 = rdm2.to(rdm1.device)
    else:
        rdm1 = rdm1.to(device)
        rdm2 = rdm2.to(device)

    rdm1_triu = extract_triu(rdm1)
    rdm2_triu = extract_triu(rdm2)

    return compare_methods[method](rdm1_triu, rdm2_triu, **method_kwargs).item()

In [314]:
y = torch.from_numpy(benchmark.response_data.to_numpy().T)
X = feature_maps_redux["Linear-1"]

X = X.to(torch.float32)
y = y.to(torch.float32)

rdm1 = calculate_rdm(y)
rdm2 = calculate_rdm(X)

for method in tqdm(compare_methods):
    print(method, ":", round(compare_rdms(rdm1, rdm2, method, device="cuda:0"), 5))

  0%|          | 0/3 [00:00<?, ?it/s]

spearman : 0.33863
pearson : 0.36446
concordance : 0.07972


### Torch Compare RDMs (RSAToolBox)

#### Whitened RDM Comparison

In [293]:
import scipy


def _extract_triu(X):
    mask = np.triu(np.ones_like(X, dtype=bool), k=1)
    return X[mask]


def _parse_input_rdms(rdm1, rdm2):
    """Gets the vector representation of input RDMs"""
    rdm1_triu = _extract_triu(rdm1)
    rdm2_triu = _extract_triu(rdm2)
    vector1 = rdm1_triu.reshape(1, -1)
    vector2 = rdm2_triu.reshape(1, -1)
    if not vector1.shape[1] == vector2.shape[1]:
        raise ValueError("rdm1 and rdm2 must be RDMs of equal shape")
    nan_idx = ~np.isnan(vector1)
    vector1_no_nan = vector1[nan_idx].reshape(vector1.shape[0], -1)
    vector2_no_nan = vector2[~np.isnan(vector2)].reshape(vector2.shape[0], -1)
    if not vector1_no_nan.shape[1] == vector2_no_nan.shape[1]:
        raise ValueError("rdm1 and rdm2 have different nan positions")
    return vector1_no_nan, vector2_no_nan, nan_idx[0]


def _row_col_indicator(row_i, col_i, n_cond):
    """writes the correct pattern for the row / column indicator matrix

    Args:
        row_indicator: row_i (numpy.ndarray)
        col_indicator: row_i (numpy.ndarray)
        n_cond (int): Number of conditions underlying the second moment
    """
    j = 0
    for i in range(n_cond):
        row_i[j : j + n_cond - i - 1, i] = 1
        np.fill_diagonal(col_i[j : j + n_cond - i - 1, i + 1 :], 1)
        j = j + (n_cond - i - 1)


def _row_col_indicator_g(n_cond):
    """generates a row and column indicator matrix for a vectorized
    second moment matrix. The vectorized version has the off-diagonal elements
    first (like in an RDM), and then appends the diagnoal.
    You can vectorize a second momement matrix G by
    np.diag(row_i@G@col_i.T) =  np.sum(col_i*(row_i@G)),axis=1)

    Args:
        n_cond (int): Number of conditions underlying the second moment

    Returns:
        row_indicator (numpy.ndarray): n_cond (n_cond-1)/2+n_cond * n_cond
        col_indicator (numpy.ndarray): n_cond (n_cond-1)/2+n_cond * n_cond
    """
    n_elem = int(n_cond * (n_cond - 1) / 2) + n_cond  # Number of elements in G
    row_i = np.zeros((n_elem, n_cond))
    col_i = np.zeros((n_elem, n_cond))
    _row_col_indicator(row_i, col_i, n_cond)
    np.fill_diagonal(row_i[-n_cond:, :], 1)
    np.fill_diagonal(col_i[-n_cond:, :], 1)
    return (row_i, col_i)


def _get_n_from_length(n):
    """
    calculates the size of the RDM from the vector length

    Args:
        **x**(np.ndarray): stack of RDM vectors (2D)

    Returns:
        int: n: size of the RDM

    """
    return int(np.ceil(np.sqrt(n * 2)))


def _cosine_cov_weighted(vector1, vector2, nan_idx=None):
    """computes the cosine angles between two sets of vectors
    weighted by the covariance
    If no covariance is given this is computed using the linear CKA,
    which is equivalent in this case and faster to compute.
    Otherwise reverts to _cosine_cov_weighted_slow.

    Args:
        vector1 (numpy.ndarray):
            first vectors (2D)
        vector1 (numpy.ndarray):
            second vectors (2D)
        sigma_k (Matrix):
            optional, covariance between pattern estimates

    Returns:
        cos (float):
            cosine angle between vectors

    """
    if nan_idx is None:
        nan_idx = np.ones(vector1[0].shape, bool)
    # Compute the extended version of RDM vectors in whitened space
    vector1_m = _cov_weighting(vector1, nan_idx)
    vector2_m = _cov_weighting(vector2, nan_idx)
    cos = _cosine(vector1_m, vector2_m)
    return cos


def _cov_weighting(vector, nan_idx):
    """Transforms an array of RDM vectors in to representation
    in which the elements are isotropic. This is a stretched-out
    second moment matrix, with the diagonal elements appended.
    To account for the fact that the off-diagonal elements are
    only there once, they are multipled by 2

    Args:
        vector (numpy.ndarray):
            RDM vectors (2D) N x n_dist

    Returns:
        vector_w:
            weighted vectors (M x n_dist + n_cond)

    """
    n_rdms, n_dist = vector.shape
    n_cond = _get_n_from_length(nan_idx.shape[0])
    vector_w = -0.5 * np.c_[vector, np.zeros((n_rdms, n_cond))]
    rowI, colI = row_col_indicator_g(n_cond)
    sumI = rowI + colI
    if np.all(nan_idx):
        # column and row means
        m = vector_w @ sumI / n_cond
        # Overall mean
        mm = np.sum(vector_w * 2, axis=1, keepdims=True) / (n_cond * n_cond)
        # subtract the column and row means and add overall mean
        vector_w = vector_w - m @ sumI.T + mm
    else:
        nan_idx_ext = np.concatenate((nan_idx, np.ones(n_cond, bool)))
        sumI = sumI[nan_idx_ext]
        # get matrix for double centering with missing values:
        sumI[n_dist:, :] /= 2
        diag = np.concatenate((np.ones((n_dist, 1)) / 2, np.ones((n_cond, 1))))
        # one line version much faster here!
        vector_w = vector_w - (
            vector_w @ sumI @ np.linalg.inv(sumI.T @ (diag * sumI)) @ (diag * sumI).T
        )
    # Weight the off-diagnoal terms double
    vector_w[:, :n_dist] = vector_w[:, :n_dist] * np.sqrt(2)
    return vector_w


def _cosine(vector1, vector2):
    """computes the cosine angles between two sets of vectors

    Args:
        vector1 (numpy.ndarray):
            first vectors (2D)
        vector1 (numpy.ndarray):
            second vectors (2D)
    Returns:
        cos (float):
            cosine angle between vectors

    """
    norm_1 = np.sqrt(np.einsum("ij,ij->i", vector1, vector1))
    norm_2 = np.sqrt(np.einsum("ij,ij->i", vector2, vector2))
    sel_1 = norm_1 > 0
    sel_2 = norm_2 > 0
    # without more indexing if all vectors are nonzero length
    if np.all(sel_1) and np.all(sel_2):
        # compute all inner products
        cos_ok = np.einsum("ij,kj->ik", vector1, vector2)
        # divide by sqrt of the inner products with themselves
        cos_ok /= norm_1.reshape((-1, 1))
        cos_ok /= norm_2.reshape((1, -1))
        return cos_ok
    # keep track of indexing if some vectors are 0
    # compute all inner products
    cos_ok = np.einsum("ij,kj->ik", vector1[sel_1], vector2[sel_2])
    # divide by sqrt of the inner products with themselves
    cos_ok /= norm_1[sel_1].reshape((-1, 1))
    cos_ok /= norm_2[sel_2].reshape((1, -1))
    cos = np.zeros((vector1.shape[0], vector2.shape[0]))
    np.putmask(cos, np.outer(norm_1 > 0, norm_2 > 0), cos_ok)
    return cos


def compare_correlation_cov_weighted(rdm1, rdm2):
    """calculates the correlations between two RDMs objects after whitening
    with the covariance of the entries

    Returns:
        numpy.ndarray: dist:
            correlations between the two RDMs

    """
    vector1, vector2, nan_idx = _parse_input_rdms(rdm1, rdm2)
    # compute by subtracting the mean and then calculating cosine similarity
    vector1 = vector1 - np.mean(vector1, 1, keepdims=True)
    vector2 = vector2 - np.mean(vector2, 1, keepdims=True)
    sim = _cosine_cov_weighted(vector1, vector2, nan_idx)
    return sim


def compare_cosine_cov_weighted(rdm1, rdm2):
    """calculates the cosine similarities between two RDMs objects

    Returns:
        numpy.ndarray: dist:
            cosine similarities between the two RDMs

    """
    vector1, vector2, nan_idx = _parse_input_rdms(rdm1, rdm2)
    sim = _cosine_cov_weighted(vector1, vector2, sigma_k, nan_idx)
    return sim

In [294]:
y = torch.from_numpy(benchmark.response_data.to_numpy().T)
X = feature_maps_redux["Linear-1"]

X = X.to(torch.float64)
y = y.to(torch.float64)

rdm1 = calculate_rdm(y).numpy()
rdm2 = calculate_rdm(X).numpy()

In [292]:
vector1, vector2, nanidx = _parse_input_rdms(rdm1, rdm2)
print(vector1.shape, vector2.shape, nanidx.shape)

_cosine_cov_weighted(vector1, vector2, None)

(1, 499500) (1, 499500) (499500,)
1 1000 499500
(500500, 1000) (500500, 1000)
1 1000 499500
(500500, 1000) (500500, 1000)


array([[0.45326715]])

### RSAToolBox CloseUp

In [326]:
def _extract_triu_(X):
    return X[np.triu(np.ones_like(X, dtype=bool), k=1)]


def calc_rdm_euclidean(data):
    """
    Args:
        dataset (rsatoolbox.data.DatasetBase):
            The dataset the RDM is computed from
        descriptor (String):
            obs_descriptor used to define the rows/columns of the RDM
            defaults to one row/column per row in the dataset
    Returns:
        rsatoolbox.rdm.rdms.RDMs: RDMs object with the one RDM
    """

    sum_sq = np.sum(data**2, axis=1, keepdims=True)
    rdm = sum_sq + sum_sq.T - 2 * np.dot(data, data.T)
    rdm = _extract_triu_(rdm) / data.shape[1]
    return rdm


def calc_rdm_mahalanobis(data, noise=None):
    """
    calculates an RDM from an input dataset using mahalanobis distance
    If multiple instances of the same condition are found in the dataset
    they are averaged.

    Args:
        noise (numpy.ndarray):
            dataset.n_channel x dataset.n_channel
            precision matrix used to calculate the RDM
            default: identity matrix, i.e. euclidean distance
    """
    if noise is None:
        return calc_rdm_euclidean(data)
    kernel = matrix @ noise @ matrix.T
    rdm = (
        np.expand_dims(np.diag(kernel), 0)
        + np.expand_dims(np.diag(kernel), 1)
        - 2 * kernel
    )
    rdm = _extract_triu_(rdm) / measurements.shape[1]
    return rdm

In [327]:
sample_data = benchmark.response_data.copy().to_numpy()[:10, :100]
calc_rdm_euclidean(sample_data).shape

(45,)

In [328]:
calc_rdm_mahalanobis(sample_data).shape

(45,)